In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn as sk

import tensorflow as tf
tf.__version__

In [ ]:
np.random.seed(10)
tf.compat.v1.set_random_seed(10)

In [ ]:
# Cargo los csv
df_local = pd.read_csv("./unified-lightcurves-local.csv").iloc[:, 1:]
df_global = pd.read_csv("./unified-lightcurves-global.csv").iloc[:, 1:]
df_stellar = pd.read_csv("./unified-stellar-params.csv").iloc[:, 1:]

display(df_local)
display(df_global)
display(df_stellar)

In [ ]:
local_fluxes = df_local.iloc[:,4:]
local_labels = df_local.iloc[:,3]

global_fluxes = df_global.iloc[:,4:]
global_labels = df_global.iloc[:,3]

stellar_infos = df_stellar.iloc[:,2:]
stellar_labels = df_stellar.iloc[:,1]

In [ ]:
# esto es porque no estas normalizado los datos estelares
x = stellar_infos.values
x_scaled = sk.preprocessing.MinMaxScaler().fit_transform(x)
stellar_infos = pd.DataFrame(x_scaled)

## Particionado de datos

In [ ]:
xl_train, xl_test, yl_train, yl_test = sk.model_selection.train_test_split(
    local_fluxes, local_labels, test_size=0.3, random_state=11)
    
xg_train, xg_test, yg_train, yg_test = sk.model_selection.train_test_split(
    global_fluxes, global_labels, test_size=0.3, random_state=11)
    
xs_train, xs_test, ys_train, ys_test = sk.model_selection.train_test_split(
    stellar_infos, stellar_labels, test_size=0.3, random_state=11)
    

In [ ]:
print(xl_train.shape, xl_test.shape, yl_train.shape, yl_test.shape)
print(xg_train.shape, xg_test.shape, yg_train.shape, yg_test.shape)
print(xs_train.shape, xs_test.shape, ys_train.shape, ys_test.shape)

In [ ]:
#Definimos las dimensiones
n_outputs = 1
nL_timesteps, nL_features  = xl_train.shape[0], xl_train.shape[1]
nG_timesteps, nG_features  = xg_train.shape[0], xg_train.shape[1]
nS_timesteps, nS_features  = xs_train.shape[0], xs_train.shape[1]

In [ ]:
print(nL_timesteps, nL_features)
print(nG_timesteps, nG_features)
print(nS_timesteps, nS_features)

In [ ]:
#Expandimos las dimensiones de train 
xle_train = np.expand_dims(xl_train,axis=-1) 
yle_train = np.array(yl_train)

xge_train = np.expand_dims(xg_train,axis=-1)
yge_train = np.array(yg_train)

xse_train = np.expand_dims(xs_train,axis=-1)
yse_train = np.array(ys_train)

In [ ]:
print(xl_train.shape, yl_train.shape)
print(xle_train.shape, yle_train.shape)

In [ ]:
#Expandimos las dimensiones de test
xle_test = np.expand_dims(xl_test,axis=-1)
yle_test = np.array(yl_test)

xge_test = np.expand_dims(xg_test,axis=-1)
yge_test = np.array(yg_test)

xse_test = np.expand_dims(xs_test,axis=-1)
yse_test = np.array(ys_test)

In [ ]:
print(xl_test.shape, yl_test.shape)
print(xle_test.shape, yle_test.shape)

## Construcción Red Neuronal

### Red para vistas locales

In [ ]:
# first input model
inputLocalView = tf.keras.layers.Input(shape=(nL_features, 1))
inputLocalView.set_shape([nL_timesteps, nL_features, 1]) # 4266 x 101

CL1 = tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu')(inputLocalView)
CL2 = tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu')(CL1)

ML1 = tf.keras.layers.MaxPooling1D(pool_size=7, strides=2)(CL2)

CL3 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu')(ML1)
CL4 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu')(CL3)

ML2 = tf.keras.layers.MaxPooling1D(pool_size=7, strides=2)(CL4)
flat1 = tf.keras.layers.Flatten()(ML2)

### Red para vistas globales

In [ ]:
inputGlobalView = tf.keras.layers.Input(shape=(nG_features, 1))
inputGlobalView.set_shape([nG_timesteps, nG_features, 1])  # 4266 x 1001


CG1 = tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu')(inputGlobalView)
CG2 = tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu')(CG1)

MG1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)(CG2)

CG3 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu')(MG1)
CG4 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu')(CG3)

MG2 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG4)

CG5 = tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu')(MG2)
CG6 = tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu')(CG5)

MG3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG6)

CG7 = tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu')(MG3)
CG8 = tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu')(CG7)

MG4 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG8)

flat2 = tf.keras.layers.Flatten()(MG4)


### Red para parámetros estelares

In [ ]:
StellarParam6PInput = tf.keras.layers.Input(shape=(nS_features, 1))
StellarParam6PInput.set_shape([nS_timesteps, nS_features, 1])  # 4266 x 15

#StellarParam6PInput=(nS_features,1)
flat3 = tf.keras.layers.Flatten()(StellarParam6PInput)


### Unificación de redes

In [280]:
# merge input models
merge = tf.keras.layers.concatenate([flat1, flat2, flat3])

# interpretation model
hidden1 = tf.keras.layers.Dense(256, activation='relu')(merge)
hidden2 = tf.keras.layers.Dense(256, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(256, activation='relu')(hidden2)
hidden4 = tf.keras.layers.Dense(256, activation='relu')(hidden3)

output = tf.keras.layers.Dense(n_outputs, activation='tanh')(hidden4)

model = tf.keras.Model(inputs=[inputLocalView,inputGlobalView,StellarParam6PInput], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "functional_48"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           [(4266, 1001, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_158 (Conv1D)             (4266, 997, 16)      96          input_44[0][0]                   
__________________________________________________________________________________________________
conv1d_159 (Conv1D)             (4266, 993, 16)      1296        conv1d_158[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_78 (MaxPooling1D) (4266, 496, 16)      0           conv1d_159[0][0]                 
______________________________________________________________________________________

In [281]:
# divisores 4266: 1, 2, 3, 6, 9, 18, 27, 54, 79, 158, 237, 474, 711, 1422, 2133, 4266
bs = 474
Ajuste = model.fit([xle_train, xge_train, xse_train], yle_train, epochs=50, batch_size=bs)

Epoch 1/50
9/9 [==============================] - 11s 1s/step - loss: nan - accuracy: 0.7583
Epoch 2/50
9/9 [==============================] - 10s 1s/step - loss: nan - accuracy: 0.7583
Epoch 3/50
3/9 [=========>....................] - ETA: 4s - loss: nan - accuracy: 0.7539

KeyboardInterrupt: 

In [257]:
y_pred_keras = model.predict([xle_test,xge_test, xse_test]).ravel()
display(y_pred_keras)

array([1.9541033e-01, 8.8378060e-01, 2.0661591e-01, ..., 7.9641171e-02,
       2.3715198e-04, 1.4054148e-01], dtype=float32)

In [258]:
# Defino las métricas
fpr_keras, tpr_keras, thresholds_keras = roc_curve(yle_test, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)
gmeans = np.sqrt(tpr_keras * (1-fpr_keras))

# localiza el índice del mayor g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds_keras[ix], gmeans[ix]))


Best Threshold=0.196187, G-Mean=0.695


# "#############"


In [ ]:
#Umbral estándar
labels_Standart = (y_pred_keras >= 0.5).astype(np.int)
PhiM_Standart = matthews_corrcoef(yle_test, labels_Standart)
Matrix_Standart = confusion_matrix(yle_test, labels_Standart)

#Imprimo por pantalla las métricas
auc_keras, PhiM_Standart, print(Matrix_Standart)


In [ ]:
#Umbral OPT
labels_OPT =(y_pred_keras >= thresholds_keras[ix]).astype(np.int)
PhiCoeff_OPT = matthews_corrcoef(yle_test, labels_OPT)
Matrix_OPT = confusion_matrix(yle_test, labels_OPT)
F_Measure_OPT = f1_score(yle_test, labels_OPT, average='binary')
Accuracy_OPT= accuracy_score(yle_test, labels_OPT, normalize=True)
Recall_OPT= recall_score(yle_test, labels_OPT, average=None)
Average_precision_OPT= average_precision_score(yle_test, labels_OPT)


#Imprimo por pantalla las métricas
print("AUC:", auc_keras)
print("F_Measure:", F_Measure_OPT)
print("PhiCoeff:", PhiCoeff_OPT)
print("Average_precision:", Average_precision_OPT)
print("Accuracy:", Accuracy_OPT)
print("Recall:", Recall_OPT)
print("Matriz de confusión:")
print( Matrix_OPT)


In [ ]:
#Ploteo la matrix de confusión
import seaborn as sn
df_cm = pd.DataFrame(Matrix_OPT, index = [i for i in "01"],
                  columns = [i for i in "01"])

plt.figure(0.5)
heat_map = sn.heatmap(df_cm, xticklabels=True, yticklabels=True, annot=True, annot_kws = {"ha": 'center',"va": 'bottom'},cbar=False)
#heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)
plt.xlabel('Predicted')
plt.ylabel('Actual')

for text in heat_map.texts:
    text.set_size(14)
    text.set_weight('bold')
    if text.get_text() == '5':
        text.set_color('black')


In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='CNN (area = {:.3f})'.format(auc_keras))
plt.scatter(fpr_keras[ix], tpr_keras[ix], marker='o', color='black', label='Best')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')


In [ ]:
y_pred_keras_tot = np.array([])
y_pred_rf_tot = np.array([])
y_test_total = np.array([])

xl_train, xl_test, yl_train, yl_test = train_test_split(LocalView.iloc[:,2:], LocalView.iloc[:,1], test_size=0.3, random_state=11)
xg_train, xg_test, yg_train, yg_test = train_test_split(GlobalView.iloc[:,2:], GlobalView.iloc[:,1], test_size=0.3, random_state=11)
xs_train, xs_test, ys_train, ys_test = train_test_split(StellarParam6P.iloc[:,2:], StellarParam6P.iloc[:,1], test_size=0.3, random_state=11)
    
   
    #Definimos las dimensiones
n_outputs = 1
nL_timesteps, nL_features  = xl_train.shape[0], xl_train.shape[1]
nG_timesteps, nG_features  = xg_train.shape[0], xg_train.shape[1]
nS_timesteps, nS_features  = xs_train.shape[0], xs_train.shape[1]
      
    #Expandimos las dimensiones
xle_train = np.expand_dims(xl_train,axis=-1) # axis=-3 para que se adapte a la forma del tensor
yle_train = np.array(yl_train) # Hay que trasponer 
xge_train = np.expand_dims(xg_train,axis=-1)
yge_train = np.array(yg_train)
xse_train = np.expand_dims(xs_train,axis=-1)
yse_train = np.array(ys_train)
    
    
    # Multiple Inputs

    # first input model
inputLocalView = Input(shape=(nL_features, 1))
inputLocalView.set_shape([nL_timesteps, nL_features, 1])

CL1 = Conv1D(filters=16, kernel_size=5, activation='relu')(inputLocalView)
CL2 = Conv1D(filters=16, kernel_size=5, activation='relu')(CL1)

ML1 = MaxPooling1D(pool_size=7, strides=2)(CL2)

CL3 = Conv1D(filters=32, kernel_size=5, activation='relu')(ML1)
CL4 = Conv1D(filters=32, kernel_size=5, activation='relu')(CL3)

ML2 = MaxPooling1D(pool_size=7, strides=2)(CL4)
flat1 = Flatten()(ML2)

    # second input model

inputGlobalView = Input(shape=(nG_features, 1))
inputGlobalView.set_shape([nG_timesteps, nG_features, 1])

CG1 = Conv1D(filters=16, kernel_size=5, activation='relu')(inputGlobalView)
CG2 = Conv1D(filters=16, kernel_size=5, activation='relu')(CG1)

MG1 = MaxPooling1D(pool_size=3, strides=2)(CG2)

CG3 = Conv1D(filters=32, kernel_size=5, activation='relu')(MG1)
CG4 = Conv1D(filters=32, kernel_size=5, activation='relu')(CG3)

MG2 = MaxPooling1D(pool_size=2, strides=2)(CG4)

CG5 = Conv1D(filters=64, kernel_size=5, activation='relu')(MG2)
CG6 = Conv1D(filters=64, kernel_size=5, activation='relu')(CG5)

MG3 = MaxPooling1D(pool_size=2, strides=2)(CG6)

CG7 = Conv1D(filters=128, kernel_size=3, activation='relu')(MG3)
CG8 = Conv1D(filters=128, kernel_size=3, activation='relu')(CG7)

MG4 = MaxPooling1D(pool_size=2, strides=2)(CG8)

flat2 = Flatten()(MG4)

    # third input model

StellarParam6PInput = Input(shape=(nS_features, 1))
StellarParam6PInput.set_shape([nS_timesteps, nS_features, 1])

    #StellarParam6PInput=(nS_features,1)
flat3 = Flatten()(StellarParam6PInput)
    # merge input models
merge = concatenate([flat1, flat2, flat3])

    # interpretation model
hidden1 = Dense(256, activation='relu')(merge)
hidden2 = Dense(256, activation='relu')(hidden1)
hidden3 = Dense(256, activation='relu')(hidden2)
hidden4 = Dense(256, activation='relu')(hidden3)
 #2output = Dense(n_outputs, activation='tanh')(hidden4)
output = Dense(n_outputs, activation='tanh')(hidden4)
model = Model(inputs=[inputLocalView, inputGlobalView,StellarParam6PInput], outputs=output)
    #loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'ce']
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    #loss_weights=['main_output': 1., 'aux_output': 0.2]) por si queremos darles pesos diferentes

batch_size=119

#Aumento número de muestras
for i in range(10):
    
    print(i)
    
    # Divido entre entrenamiento y test
    xl_train, xl_test, yl_train, yl_test = train_test_split(LocalView.iloc[:,2:], LocalView.iloc[:,1], test_size=0.3, random_state=11+i)
    xg_train, xg_test, yg_train, yg_test = train_test_split(GlobalView.iloc[:,2:], GlobalView.iloc[:,1], test_size=0.3, random_state=11+i)
    xs_train, xs_test, ys_train, ys_test = train_test_split(StellarParam6P.iloc[:,2:], StellarParam6P.iloc[:,1], test_size=0.3, random_state=11+i)
    
  
    # fit forma 2
    FIT = model.fit([xle_train, xge_train, xse_train], yle_train,
          epochs=50, batch_size=batch_size, verbose=0)
    
    xle_test = np.expand_dims(xl_test,axis=-1)
    yle_test = np.array(yl_test) 
    xge_test = np.expand_dims(xg_test,axis=-1)
    yge_test = np.array(yg_test)
    xse_test = np.expand_dims(xs_test,axis=-1)
    yse_test = np.array(ys_test)
    
    
    # Creo la RoC
    y_pred_kerasC = model.predict([xle_test,xge_test,xse_test]).ravel()
    
    
    #Lleno los vectores totales
    y_pred_keras_tot = np.concatenate((y_pred_keras_tot,y_pred_kerasC))
    
    y_test_total = np.concatenate((y_test_total,yle_test))


In [ ]:
# Defino las métricas

fpr_kerasTotal, tpr_kerasTotal, thresholds_kerasTotal = roc_curve(y_test_total, y_pred_keras_tot)
auc_kerasTotal = auc(fpr_kerasTotal, tpr_kerasTotal)
gmeans = np.sqrt(tpr_kerasTotal * (1-fpr_kerasTotal))
# localiza el índice del mayor g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds_kerasTotal[ix], gmeans[ix]))


In [ ]:
#Umbral estándar
labels_Standart = (y_pred_keras_tot >= 0.5).astype(np.int)
PhiCoeff_Standart = matthews_corrcoef(y_test_total, labels_Standart)
Matrix_Standart = confusion_matrix(y_test_total, labels_Standart)
F_Measure_Standart = f1_score(y_test_total, labels_Standart, average='binary')

#Imprimo por pantalla las métricas
auc_kerasTotal, PhiM_Standart,F_Measure_Standart, print(Matrix_Standart)


In [ ]:
#Umbral OPT
labels_OPT =(y_pred_keras_tot >= thresholds_kerasTotal[ix]).astype(np.int)
PhiCoeff_OPT = matthews_corrcoef(y_test_total, labels_OPT)
Matrix_OPT = confusion_matrix(y_test_total, labels_OPT)
F_Measure_OPT = f1_score(y_test_total, labels_OPT, average='binary')
Accuracy_OPT= accuracy_score(y_test_total, labels_OPT, normalize=True)
Recall_OPT= recall_score(y_test_total, labels_OPT, average=None)
Average_precision_OPT= average_precision_score(y_test_total, labels_OPT)


#Imprimo por pantalla las métricas
print("AUC:", auc_kerasTotal)
print("F_Measure:", F_Measure_OPT)
print("PhiCoeff:", PhiCoeff_OPT)
print("Average_precision:", Average_precision_OPT)
print("Accuracy:", Accuracy_OPT)
print("Recall:", Recall_OPT)
print("Matriz de confusión:")
print( Matrix_OPT)


In [ ]:
#Ploteo la matrix de confusión

df_cm_OPT = pd.DataFrame(Matrix_OPT, index = [i for i in "01"],
                  columns = [i for i in "01"])
plt.figure(0.5)
heat_map = sn.heatmap(df_cm_OPT, xticklabels=True, yticklabels=True, annot=True, fmt='g', annot_kws = {"ha": 'center',"va": 'bottom'},cbar=False)
#heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)
plt.xlabel('Predicted')
plt.ylabel('Actual')

for text in heat_map.texts:
    text.set_size(14)
    text.set_weight('bold')
    if text.get_text() == '33':
        text.set_color('black')


In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_kerasTotal, tpr_kerasTotal,  color='green', label='Tanh/Adadelta = {:.3f})'.format(auc_kerasTotal))
plt.scatter(fpr_kerasTotal[ix], tpr_kerasTotal[ix], marker='o', color='black', label='Best')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()